In [1]:
# if you want to use on COLAB upload PDF files to your drive and set patth correctly my folder is "pdfs"
# else commend this cell
from google.colab import drive
drive.mount('/content/drive')
!cp drive/MyDrive/pdfs/test.pdf test.pdf

Mounted at /content/drive


In [2]:
!pip install pytesseract pdf2image langdetect
!pip install opencv-python==4.5.1.48
!sudo apt install tesseract-ocr
!apt-get install poppler-utils
!apt-get install tesseract-ocr-fra

     |████████████████████████████████| 983kB 16.7MB/s 
  Created wheel for pytesseract: filename=pytesseract-0.3.7-py2.py3-none-any.whl size=13945 sha256=8e5d854916cdea33cf29fbf05537efb96af6c6f284f0f5f9844d9be91a855289
  Stored in directory: /root/.cache/pip/wheels/81/20/7e/1dd0daad1575d5260916bb1e9781246430647adaef4b3ca3b3
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl size=993193 sha256=40a380b1c4c074cfa34e90a39c504815d4ab3de60a67f407b79ce7d4a02d2ae0
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built pytesseract langdetect
     |████████████████████████████████| 50.4MB 65kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
Reading package lists... Done
Building de

In [3]:
from pdf2image import convert_from_path 
from pytesseract import Output 
from langdetect import detect
from PIL import Image 
import pandas as pd
import numpy as np
import pytesseract 
import sys 
import cv2
import os 
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

In [4]:
# !ls

In [5]:
def pdf_to_images(PDF_file) :
    pages = convert_from_path(PDF_file, 300)  # DPI
    image_counter = 1
    for page in pages:
      filename = PDF_file+"_"+str(image_counter)+".jpg"
      page.save(filename, 'JPEG') 
      image_counter += 1
    return pages

def is_french(stringx) :
    try :
        if detect(stringx.lower()) == 'fr' :
            return True
    except :
        return False

In [6]:
################################# We can adapt this code to pipeline############################################
        # text_list = text_to_list(text)
        # for line in text_list :
        #   if is_french(line) or line.startswith('Artic') or line.startswith('§')  :
        #     # if is_french(line) or line.startswith('Artic')  :
        #     fr_list.append(line.replace('\n',' '))

In [7]:
##### Functions for the Pipeline start here #####
# 1 - call English NLP and store results to a variable (because it finds § symbol perfectly)
def en_nlp_for_symbol(filename):  
  d = pytesseract.image_to_data(filename, output_type=Output.DICT)
  return d.copy()

In [8]:
# 2 - call French NLP and store results to another variable
def fr_nlp_for_symbol(filename):  
  d = pytesseract.image_to_data(filename, output_type=Output.DICT, lang="fra")
  return d.copy()

In [9]:
### if you like you can SAVE and SEE boxed PDF image for fr_nlp and en_nlp USE this code ###
### n_boxes = len(d['level'])
### for i in range(n_boxes):
###   if(d['text'][i] != ""):
###     (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
###     cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
### cv2.imwrite('orhan2.png', img) ###

In [10]:
# 3 - from English NLP results, system collect all § symbols coordinates and index info in a list
def info_from_en_nlp_symbol(en_nlp, symbol="§"):
  pos_list=[]         # list has records in a tuple (index of English NLP, [coordinates,top,left,...])    
  for t in range(len(en_nlp['text'])):
    if symbol in en_nlp['text'][t]:
      pos_list.append([t,[en_nlp['left'][t], en_nlp['top'][t], en_nlp['width'][t], en_nlp['height'][t]]])
  return pos_list

In [11]:
# 4 - match all § symbols coordinates with French NLP results and make a list
def match_lists_en_and_fr_nlps(fr_nlp,pos_list, tolerans=1):   # give one pixel tolerans to match Fr and En NLPs
  match_list=[]  # x[0] = left ,  x[1] = top , x[2] = width , x[3] = height ,
  for k in pos_list:#for k,(a,x) in enumerate(pos_list):
    for l0 in range(len(fr_nlp["left"])):  
      if ( fr_nlp["left"][l0] + tolerans > k[1][0] ) and ( fr_nlp["left"][l0] - tolerans < k[1][0] ):
        if ( fr_nlp["top"][l0] + tolerans > k[1][1] ) and ( fr_nlp["top"][l0] - tolerans < k[1][1] ):
          if ( fr_nlp["width"][l0] + tolerans > k[1][2] ) and ( fr_nlp["width"][l0] - tolerans < k[1][2] ):
            if ( fr_nlp["height"][l0] + tolerans > k[1][3] ) and ( fr_nlp["height"][l0] - tolerans < k[1][3] ):            
              match_list.append([l0,[fr_nlp['left'][l0], fr_nlp['top'][l0], fr_nlp['width'][l0], fr_nlp['height'][l0]]])#print(fr_nlp["text"][l0]," , index = ",l0,fr_nlp["left"][l0],fr_nlp["top"][l0],fr_nlp["width"][l0],fr_nlp["height"][l0])
  return match_list

In [12]:
# 5 - Delete all duplicates from matched boxes list
def delete_dublicates_from_list(match_list):
  delete_list=[]
  for k in match_list:
    cont_list=[]
    for k1 in match_list:
      if k[1]==k1[1]:
        cont_list.append(k1[0])
    for element in cont_list:
      if max(cont_list)!= element: 
        delete_list.append(element)
  for d in list(dict.fromkeys(delete_list)):
    for m in match_list:
      if m[0] == d:
        match_list.pop(match_list.index(m))  # print(len(match_list),match_list)
  return match_list

In [13]:
# 7 - call is French function to filter to get just French results
def df_fr_sentences(df_fr):
  df_fr_filtered = pd.DataFrame()
  df_fr_filtered["id"]=list(df_fr.index.values)
  df_fr_filtered["fullsentence"] =df_fr.groupby(['block_num','par_num','line_num'])['text'].transform(' '.join)
  for i in range(20):
    df_fr_filtered["fullsentence"] = df_fr_filtered["fullsentence"].replace(["      ","     ","    ","   ","  "],"")
  df_fr_filtered = df_fr_filtered.drop(df_fr_filtered[df_fr_filtered["fullsentence"] == ""].index , axis=0)
  df_fr_filtered = df_fr_filtered.drop(df_fr_filtered[df_fr_filtered["fullsentence"] == " "].index , axis=0)
  nl_sen =pd.Series([is_french(x) for x in df_fr_filtered["fullsentence"]])
  return df_fr_filtered.loc[nl_sen.fillna(False).values]

In [14]:
### PIPELINE OF MERGING TWO NLP RESULTS TO BE ABLE TO GET FRENCH CHARACTERSET AND § SYMBOL

def fill_fr_nlp_from_en_nlp(image_name):
  # 1 - call English NLP and store results to a variable (because it finds § symbol perfectly)
  en_nlp = en_nlp_for_symbol(image_name) # 300 dpi is recommended for OCR image files.
  df_en = pd.DataFrame(en_nlp)  

  # 2 - call French NLP and store results to another variable
  fr_nlp = fr_nlp_for_symbol(image_name)
  df_fr = pd.DataFrame(fr_nlp)  

  # 3 - from English NLP results, system collect all § symbols coordinates and index info in a list
  pos_en = info_from_en_nlp_symbol(en_nlp)

  # 4 - match all § symbols coordinates with French NLP results and make a list
  pos_fr = match_lists_en_and_fr_nlps(fr_nlp, pos_en)

  # 5 - Delete all duplicates from match list
  pos_fr = delete_dublicates_from_list(pos_fr)

  # 6 - Change French results with desired characters
  for m in pos_fr:
    df_fr["text"][m[0]] = df_en["text"][pos_en[pos_fr.index(m)][0]]

  # 7 - call is French function to filter to get just French results
  df_fr_sent = df_fr_sentences(df_fr)
  
  df_fr["id"] = df_fr.index.values
  df_fr = pd.merge(df_fr, df_fr_sent, how='inner', on=['id'])
  
  return df_fr#, df_fr_sent #, df_en , match_list, pos_list

In [15]:
pages = pdf_to_images("test.pdf") # this cell and next cell can merge to have a function

In [16]:
df_fr_full = pd.DataFrame(columns = ['id','level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])
print(f"this document has {len(pages) + 1} pages process is started...")
for page in pages:
  df_fr = fill_fr_nlp_from_en_nlp(pages[0])  
  del df_fr["page_num"]   
  df_fr["page_num"] = pages.index(page) + 1 
  df_fr_full = df_fr_full.append(df_fr)
  print(f"{pages.index(page) + 1}. page is finished")
df_fr_full = df_fr_full.reset_index(drop=True)

this document has 8 pages process is started...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1. page is finished
2. page is finished
3. page is finished
4. page is finished
5. page is finished
6. page is finished
7. page is finished


In [19]:
df_fr_full

,id,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,fullsentence
0,22,4,1,4,1,1,0,621,354,302,32,-1,,COLLECTIEVE
1,23,5,1,4,1,1,1,621,354,302,32,90,COLLECTIEVE,COLLECTIEVE
2,37,5,1,4,2,1,3,800,529,202,32,91,PARITAIR,HET AANVULLENDE PARITAIR COMITE
3,114,4,1,6,1,1,0,1307,354,851,33,-1,,CONVENTION COLLECTIVE DE TRAVAIL
4,115,5,1,6,1,1,1,1307,354,286,33,95,CONVENTION,CONVENTION COLLECTIVE DE TRAVAIL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1780,659,5,7,30,2,7,1,1395,3182,45,33,96,du,du régime de chômage avec
1781,660,5,7,30,2,7,2,1457,3182,130,41,96,régime,du régime de chômage avec
1782,661,5,7,30,2,7,3,1602,3183,46,33,95,de,du régime de chômage avec
1783,662,5,7,30,2,7,4,1664,3183,177,41,95,chômage,du régime de chômage avec


In [18]:
##### now it is turn for this 5 last tasks #####
# 1 - call structuring functions to add new columns to the dataset      # Almost finished
# 2 - call storeDB function to save everything to the DB                # a few lines
# 3 - call function to update DB with CLA relations and changes         # a few lines
# 4 - call QnA system to save desired questions and answers to the DB   # a few lines
# 5 - call API to present the project to the user                       # will take time
# 6 - Presentation                                                      # will take time because you did greate job guys show it